In [10]:
# Cell 2 – imports & paths

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nibabel as nib
from nilearn import image, masking
from nilearn.glm.first_level import glover_hrf

# --- Paths ---
PROJECT_ROOT = Path(".")
DATA_ROOT    = PROJECT_ROOT / "data"
DERIV_ROOT   = PROJECT_ROOT / "derivatives" / "music"

# List of all subjects
SUBJECTS = ["sub-001", "sub-002", "sub-003", "sub-004", "sub-005"]
run = 2  # we are using run-02

print("Building design matrices for all subjects...")
print()

Building design matrices for all subjects...



In [11]:
# Cell 3 – Build and save design matrices for all subjects (loop)

def build_design_matrix(bold_path: Path, design_path: Path, tr: float):
    """
    Build a simple music design matrix with HRF and save it.
    """
    img = image.load_img(bold_path)
    shape = img.shape
    n_scans = shape[-1]
    frame_times = np.arange(n_scans) * tr

    stim_music = np.ones(n_scans)
    time_length = 32
    hrf = glover_hrf(tr, oversampling=1, time_length=time_length)
    music_hrf = np.convolve(stim_music, hrf)[:n_scans]

    design = pd.DataFrame(
        {
            "const": np.ones(n_scans),
            "linear_drift": np.linspace(-1, 1, n_scans),
            "music_boxcar": stim_music,
            "music_hrf": music_hrf,
        },
        index=frame_times,
    )

    design.to_csv(design_path)
    return design


# Loop over all subjects and build/save designs
for sub_id in SUBJECTS:
    # Find BOLD file
    pattern = f"music/{sub_id}_task-*_run-0{run}_bold.nii*"
    bold_candidates = sorted(DATA_ROOT.glob(pattern))
    
    if not bold_candidates:
        print(f"[{sub_id}] WARNING: No BOLD file found – skipping.")
        continue
    
    bold_path = bold_candidates[0]
    design_path = DERIV_ROOT / f"{sub_id}_run-0{run}_design.csv"

    # Load one BOLD file to get TR and n_scans
    img = image.load_img(bold_path)
    tr = img.header.get_zooms()[3]

    if design_path.exists():
        print(f"[{sub_id}] Design already exists: {design_path}")
    else:
        print(f"[{sub_id}] Building design matrix for {bold_path.name}...")
        design = build_design_matrix(bold_path, design_path, tr)
        print(f"[{sub_id}] Saved to: {design_path}")

print("\nFinished building design matrices for all subjects.")

[sub-001] Design already exists: derivatives\music\sub-001_run-02_design.csv
[sub-002] Building design matrix for sub-002_task-Test_run-02_bold.nii...
[sub-002] Saved to: derivatives\music\sub-002_run-02_design.csv
[sub-003] Building design matrix for sub-003_task-Test_run-02_bold.nii...
[sub-003] Saved to: derivatives\music\sub-003_run-02_design.csv
[sub-004] Building design matrix for sub-004_task-Test_run-02_bold.nii...
[sub-004] Saved to: derivatives\music\sub-004_run-02_design.csv
[sub-005] Building design matrix for sub-005_task-Test_run-02_bold.nii...
[sub-005] Saved to: derivatives\music\sub-005_run-02_design.csv

Finished building design matrices for all subjects.


In [14]:
# Cell 4 – Build simple music design matrix and save to CSV (if not already built)

if design is not None:
    print("Design already loaded for", sub_id, "- nothing to build.")
else:
    print(f"Building design matrix for {sub_id} and saving to CSV...")

    # 1) Basic timing info
    n_scans = shape[-1]
    frame_times = np.arange(n_scans) * tr

    # 2) Music boxcar: assume music is on for the whole run
    stim_music = np.ones(n_scans)

    # 3) HRF (Glover) and convolve
    from nilearn.glm.first_level import glover_hrf

    time_length = 32
    hrf_times = np.arange(0, time_length, tr)
    hrf = glover_hrf(tr, oversampling=1, time_length=time_length)

    music_hrf = np.convolve(stim_music, hrf)[:n_scans]

    # 4) Simple design matrix
    design = pd.DataFrame(
        {
            "const": np.ones(n_scans),
            "linear_drift": np.linspace(-1, 1, n_scans),
            "music_boxcar": stim_music,
            "music_hrf": music_hrf,
        },
        index=frame_times,
    )

    print("Design columns:", list(design.columns))
    print("Design shape  :", design.shape)

    # 5) Save it
    design.to_csv(design_path)
    print("Saved design to:", design_path)

Design already loaded for sub-005 - nothing to build.
